Построение модели нейронной сети
===================

Нейронные сети состоят из слоев/модулей, которые выполняют операции с данными.

Пространство имен `torch.nn` https://pytorch.org/docs/stable/nn.html из библиотеки `torch` содержит всё необходимое для того, чтобы построить свою нейронную сеть.

Каждый модуль в Pytorch является подклассом `nn.Module` https://pytorch.org/docs/stable/generated/torch.nn.Module.html


Нейронная сеть - это сам module, состоящий из других модулей (слоев). Такая вложенная структура позволяет легко создавать сложные архитектуры и управлять ими. 

В следующих разделах мы построим нейронную сеть для классификации изображений в датасете FashionMNIST.

In [4]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

Выбор устройства 
-----------------------

Мы хотим иметь возможность обучать нашу модель на аппаратном ускорителе, таком как GPU, если он доступен.

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


Определение класса
-------------------------
Мы определяем нашу нейронную сеть путем наследования от класса `nn.Module` и инициализируем слои нейронной сети в `__init__`. Каждый потом класса `nn.Module` реализует операции над входными данными в методе `forward`.

In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Мы создаем экземпляр класса `Neuralnetwork`, перемещаем его на GPU и распечатываем структуру сети.

In [7]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Чтобы использовать модель, мы передаем ей входные данные. Это действие запускает функцию ``forward`` вместе с некоторыми фоновыми операциями. Подробнее здесь: https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866

Не нужно вызывать ``model.forward()`` напрямую!

На выходе возвращает 10-мерный тензор с необработанными предсказанными значениями для каждого класса.

Мы получаем вероятности предсказания, пропуская их через экземпляр модуля ``nn.Softmax``.


In [8]:
# картинка
X = torch.rand(1, 28, 28, device=device)
# загрузили картинку в модель и получили предсказания по каждому классу
logits = model(X)
logits

tensor([[ 0.0232,  0.0358, -0.0801, -0.0514, -0.0151,  0.0626,  0.0246,  0.0219,
         -0.0701, -0.0494]], device='cuda:0', grad_fn=<AddmmBackward>)

In [9]:
pred_probab = nn.Softmax(dim=1)(logits)
pred_probab

tensor([[0.1032, 0.1046, 0.0931, 0.0958, 0.0994, 0.1074, 0.1034, 0.1031, 0.0940,
         0.0960]], device='cuda:0', grad_fn=<SoftmaxBackward>)

In [10]:
pred_probab.sum()

tensor(1., device='cuda:0', grad_fn=<SumBackward0>)

In [11]:
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred[0]}")

Predicted class: 5


--------------




Слои модели нейронной сети
-------------------------

Давайте разберем слои в модели FashionMNIST. Чтобы проиллюстрировать это, мы возьмем батч из 3 изображений размером 28х28 и посмотрим, что с ним произойдет, когда мы пропустим его через сетку.

In [12]:
input_image_batch = torch.rand(3,28,28)
print(input_image_batch.size())

torch.Size([3, 28, 28])


#### 1. nn.Flatten() 

https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html

We initialize the `nn.Flatten  <https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html>`_
layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values (
the minibatch dimension (at dim=0) is maintained).

Мы инициализируем слой `nn.Flatten` для преобразования каждого 2D-изображения 28х28 в непрерывный массив из 784 значений пикселей (батч размер при dim=0 сохраняется)

$$28 \cdot 28 = 784$$

In [14]:
flatten = nn.Flatten()
flat_image_batch = flatten(input_image_batch)
print(flat_image_batch.size())

torch.Size([3, 784])


#### 2. nn.Linear() 

https://pytorch.org/docs/stable/generated/torch.nn.Linear.html

`linear layer` - это модуль, который применяет линейное преобразование к входным данным (), используя сохраненные веса и смещения.

In [17]:
layer_one = nn.Linear(in_features=28*28, out_features=20)
hidden_one = layer_one(flat_image_batch)
print(hidden_one.size())
print(hidden_one)

torch.Size([3, 20])
tensor([[ 0.3665,  0.0361,  0.0930,  0.2152,  0.0112,  0.1880,  0.2241, -0.2491,
         -0.6050, -0.1149,  0.0077, -0.1558, -0.5064, -0.2857, -0.2840, -0.3226,
          0.0915,  0.8398, -0.4459, -0.1764],
        [ 0.4147, -0.0458,  0.2696,  0.2572,  0.4085, -0.1462,  0.2459, -0.1277,
         -0.6242, -0.1008,  0.4359, -0.0393, -0.6298, -0.2026, -0.0579, -0.1220,
         -0.1433,  0.6212, -0.2644, -0.2427],
        [ 0.1642, -0.4136,  0.2468,  0.5216, -0.1032,  0.0945,  0.7037,  0.0044,
         -0.9589,  0.1309,  0.2551,  0.2022, -0.5262, -0.4445, -0.3528, -0.4342,
         -0.1213,  1.0457, -0.3880, -0.2877]], grad_fn=<AddmmBackward>)


#### 3. nn.ReLU()

https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html

Нелинейные активации создают сложные сопоставления между входными и выходными данными модели. Они применяются после линейных преобразований для введения *нелинейности*, помогая нейронным сетям изучать широкий спектр явлений.

В данной модели мы используем `nn.ReLU()` между линейными слоями, но есть и другие активации для внесения нелинейности в нашу модель.

In [19]:
print(f"Before ReLU: {hidden_one}\n\n")
hidden_one = nn.ReLU()(hidden_one)
print(f"After ReLU: {hidden_one}")

Before ReLU: tensor([[ 0.3665,  0.0361,  0.0930,  0.2152,  0.0112,  0.1880,  0.2241, -0.2491,
         -0.6050, -0.1149,  0.0077, -0.1558, -0.5064, -0.2857, -0.2840, -0.3226,
          0.0915,  0.8398, -0.4459, -0.1764],
        [ 0.4147, -0.0458,  0.2696,  0.2572,  0.4085, -0.1462,  0.2459, -0.1277,
         -0.6242, -0.1008,  0.4359, -0.0393, -0.6298, -0.2026, -0.0579, -0.1220,
         -0.1433,  0.6212, -0.2644, -0.2427],
        [ 0.1642, -0.4136,  0.2468,  0.5216, -0.1032,  0.0945,  0.7037,  0.0044,
         -0.9589,  0.1309,  0.2551,  0.2022, -0.5262, -0.4445, -0.3528, -0.4342,
         -0.1213,  1.0457, -0.3880, -0.2877]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.3665, 0.0361, 0.0930, 0.2152, 0.0112, 0.1880, 0.2241, 0.0000, 0.0000,
         0.0000, 0.0077, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0915, 0.8398,
         0.0000, 0.0000],
        [0.4147, 0.0000, 0.2696, 0.2572, 0.4085, 0.0000, 0.2459, 0.0000, 0.0000,
         0.0000, 0.4359, 0.0000, 0.0000, 0.0000, 0.000

#### 4. nn.Sequential()

https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html

`nn.Sequential` - это упорядоченный контейнер модулей. Данные проходят через все модули (слои) в том порядке, в котором они определены. Вы можете использовать последовательные контейнеры для создания быстрой сети, такой как ``seq_modules``

In [23]:
seq_modules = nn.Sequential(
    flatten, 
    layer_one,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)
logits

tensor([[-9.2316e-02,  1.4551e-01,  1.8266e-01, -2.1296e-01,  2.4451e-01,
         -1.1371e-01, -1.5631e-04, -2.2041e-02,  1.1919e-01,  1.1718e-02],
        [-4.9443e-02,  1.0962e-01,  1.1151e-01, -1.3771e-01,  2.0328e-01,
         -1.7800e-01, -1.4323e-01,  6.6048e-02,  6.1901e-02,  4.9397e-02],
        [-2.0679e-01,  7.8264e-02,  6.9283e-02,  6.9746e-04,  5.1431e-02,
         -2.9739e-01, -1.8378e-01,  1.3752e-01,  9.2133e-02, -5.1897e-02]],
       grad_fn=<AddmmBackward>)

#### 5. nn.Softmax()

https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html

Последний линейный слой нейронной сети возвращает `logits` - необработанные значения в диапазоне $(-\infty; +\infty)$ передаются в `nn.Softmax()`. Что это значит?

`logits` масштабируются к значениям $[0; 1]$, которые будут представлять предсказания модели для каждого класса в виде вероятности. Если сложить все значения, то должна получаться $1$. Параметр `dim` указывает измерение, по которому сумма значений должна равняться единице.

In [25]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
pred_probab

tensor([[0.0880, 0.1116, 0.1158, 0.0780, 0.1232, 0.0861, 0.0965, 0.0944, 0.1087,
         0.0976],
        [0.0936, 0.1097, 0.1099, 0.0857, 0.1205, 0.0823, 0.0852, 0.1051, 0.1046,
         0.1033],
        [0.0831, 0.1105, 0.1095, 0.1022, 0.1076, 0.0759, 0.0850, 0.1172, 0.1120,
         0.0970]], grad_fn=<SoftmaxBackward>)

#### 6. Предсказание класса для каждой картинки в батче

In [28]:
for i in range(3):
    pred = pred_probab[i].argmax(0)
    print(f"Изображение №{i+1}, Предсказание: {pred} класс")

Изображение №1, Предсказание: 4 класс
Изображение №2, Предсказание: 4 класс
Изображение №3, Предсказание: 7 класс


Параметры модели 
-------------------------

Многие слои внутри нейронной сети *параметризованы*, то есть имеют связанные веса и смещения, которые оптимизируются во время обучения. 

Подкласс ``nn.Module`` автоматически отслеживает все поля, определенные внутри объекта вашей модели, и делает все параметры доступными с помощью методов вашей модели ``parameters()`` или ``named_parameters()``.


В этом примере мы перебираем каждый параметр, затем печатаем его размер и предварительное значение.

In [35]:
print(f"Model structure: {model}")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [36]:
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0197, -0.0017, -0.0034,  ...,  0.0045, -0.0024,  0.0304],
        [ 0.0355,  0.0353,  0.0172,  ...,  0.0060,  0.0015,  0.0060]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0025,  0.0193], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0110,  0.0262, -0.0165,  ..., -0.0339,  0.0170,  0.0426],
        [-0.0350, -0.0407,  0.0196,  ..., -0.0384,  0.0387, -0.0232]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.bias | Size: torch.Size([512]) | Values : tensor([0.0234, 0.0287], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.4.weight | Size: torch.Size([10, 512]) | Values : tensor([[ 0.0306,  0.0327,  0.0165,  ..., -0.0096, -0.0032, -0.0142],
        [ 0.0235,  0.0324,  0.0161,  ...,

--------------




Дальнейшее чтение
--------------
- `torch.nn` API https://pytorch.org/docs/stable/nn.html



---